**Внимание!**
Если вы используете Google Collab для взаимодействия с проектом, обязательно запустите ячейку ниже для установки репозитория и необходимых библиотек. Спасибо!

In [ ]:
!git clone -l -s https://github.com/sp1dzsaus/zverushkas cloned-repo
%cd cloned-repo
!ls
!pip install -r requirements.txt

In [ ]:
from bio import AnimalDraw, AnimalGenerator

generator = AnimalGenerator(leg_count=2, length=20, straightness=0.15, gradation=100,
                            gibbosity=10, distribution=(2, 10), spreadvalue=(10, 20))

animal = generator.generate()
AnimalDraw(animal).save_3d('result.stl')


# **Зверюшки**

In [ ]:
import os
os.makedirs('output/')

## Параметрический конструктор существ

- `seed` - ключ генератора псевдослучайных чисел. Определяет общую форму животного.
- `length` - количество позвонков животного.
- `gradation` - монотонность длин позвонков.
- `straightness` - прямота животного, монотонность углов позвонков.
- `distribution` - монотонность толщины животного.
- `leg_count` - количество пар лап.
- `ground` - уровень высоты поверхности.

In [ ]:
from bio import Animal, AnimalDraw
from ipywidgets import interact

def animal_maker(seed, length, gradation, straightness, gibbosity, distribution, spreadvalue, leg_count, ground):
    global DRAW
    global ANIMAL
    ANIMAL = Animal.from_params(length, gradation / 10, straightness / 50, distribution / 10, spreadvalue / 10, gibbosity / 10, leg_count, seed)
    DRAW = AnimalDraw(ANIMAL)
    DRAW.set_ground_level(1 - ground)
    img1 = DRAW.draw(150)
    img2 = DRAW.draw_from_above(150)
    h = img1.height
    img1 = img1.crop((0, 0, img1.width, img1.height + img2.height))
    img1.paste(img2, (0, h))
    return img1


interact(animal_maker, seed=(1, 999999), length=(7, 15),
         gradation=(2.0, 500.), straightness=(1.0, 500.0), gibbosity=(50.0, 500.0), distribution=(50.0, 500.0), spreadvalue=(50.0, 500.0),
         leg_count=(1, 4), ground=(-1.00, 2.00))


### Импорт животного в 3D модель STL

In [ ]:
DRAW.save_3d('output/generated.stl')

## Алгебра животных

In [ ]:
from random import randint
from bio import Animal, AnimalDraw
from generators import *

seed = randint(0, 999999)

print('Общий сид:', seed)

In [ ]:
animal1 = Default(seed, paramseed=randint(0, 999999))
draw = AnimalDraw(animal1)
draw.save_3d('output/father.stl')
draw.draw(150)

In [ ]:
animal2 = Default(seed, paramseed=randint(0, 999999))
draw = AnimalDraw(animal2)
draw.save_3d('output/mother.stl')
draw.draw(150)

### Интерполяция животных

Плавный переход параметров из одного животного к другому.
Может использоваться для роста конкретной особи или эволюции вида.

In [ ]:
from ipywidgets import interact
from extra_maths import lerp, smoothstep

child = None

def animal_interpolator(percent):
    global child
    animal3 = Animal.from_array(seed, lerp(animal1.to_array(), animal2.to_array(), smoothstep(percent / 100)))
    child = animal3
    return AnimalDraw(animal3).draw(150)


interact(animal_interpolator, percent=(0.00, 100.00))

In [ ]:
draw = AnimalDraw(child)
draw.save_3d('output/child.stl')

### Скрещивание

Случайное перемешивание параметров между двумя животными.

In [ ]:
import numpy as np
from random import randint

a = animal1.to_array()
b = animal2.to_array()
c = np.zeros(7)

for i in range(7):
    if randint(0, 1):
        c[i] = a[i]
    else:
        c[i] = b[i]

animal4 = Animal.from_array(seed, c)
draw = AnimalDraw(animal4)
draw.save_3d('output/child.stl')
draw.draw(150)